# dspy playbook

[DSPy](https://dspy.ai/) is:
> a declarative framework for building modular AI software. It allows you to iterate fast on structured code, rather than brittle strings, and offers algorithms that compile AI programs into effective prompts and weights for your language models, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.

As such, it shifts your focus from fiddling with prompt strings to programming with structured and declarative natural-language modules. 

### Configuration

In [1]:
import os
import dspy

lm = None

openai_key = os.environ.get('OPENAI_API_KEY')
anthropic_key = os.environ.get('ANTHROPIC_API_KEY')
perplexity_key = os.environ.get('PERPLEXITY_API_KEY')
groq_key = os.environ.get('GROQ_API_KEY')

if None in [openai_key,anthropic_key,perplexity_key,groq_key]:
    raise ValueError('No supported API key found in environment variables.')

if openai_key:
    openai_lm = dspy.LM('openai/gpt-4o-mini', api_key=openai_key)
if anthropic_key:
    anthropic_lm = dspy.LM('anthropic/claude-opus-4-20250514', api_key=anthropic_key)
if perplexity_key:
    perplexity_lm = dspy.LM('perplexity/sonar', api_key=perplexity_key)
if groq_key:
    groq_lm = dspy.LM('groq/llama-3.1-8b-instant', api_key=groq_key)

dspy.configure(lm=openai_lm)

### Use Case 1: Information Extraction 

In [2]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today.  The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Inc. Announces iPhone 14
['Announcement', 'CEO Statement', 'New Features']
[{'name': 'Apple Inc.', 'type': 'Company'}, {'name': 'iPhone 14', 'type': 'Product'}, {'name': 'Tim Cook', 'type': 'Person'}]


### Use Case 2: Prediction 

In [3]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.85
)

### Use Case 3: RAG

In [4]:
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

rag = dspy.ChainOfThought("context, question -> response")

question = "What's the name of the castle that David Gregory inherited?"
rag(context=search_wikipedia(question), question=question)

Prediction(
    reasoning='David Gregory inherited Kinnairdy Castle in 1664, as mentioned in the context provided.',
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'
)

### Use Case 4: ReAct-based Agents

[ReAct](https://www.promptingguide.ai/techniques/react) is short for **Reasoning and Acting**.  It is a paradigm for AI agent design where an agent uses chain-of-thought reasoning and tool-using actions in aggregation.  With ReAct:
> LLMs are used to generate both reasoning traces and task-specific actions in an interleaved manner.

In [5]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

target = 9362158
pred = react(question="What is the year of birth of David Gregory of Kinnairdy castle?")
print(f"Reasoning: {pred.reasoning}")
yearOfBirth = pred.answer
print(yearOfBirth)

Reasoning: David Gregory was born on December 20, 1625, which provides the specific year of his birth needed to answer the question.
1625.0


In [6]:
pred = react(question=f"What is {target} divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.reasoning)
print(f"Reasoning: {pred.reasoning}")
print(pred.answer)
print(f"Expected answer is {target/yearOfBirth}")

To find the answer, I need to divide 9362158 by the year of birth of David Gregory, which is 1625. Performing the division gives me the result of 5765.0.
Reasoning: To find the answer, I need to divide 9362158 by the year of birth of David Gregory, which is 1625. Performing the division gives me the result of 5765.0.
5765.0
Expected answer is 5761.328


### Use Case 5: Multi-Stage Pipelines

In [7]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="Current state of Quantum Computing and Outlook for the next few years")

In [8]:
from IPython.display import display, Markdown

for section in article.sections:
    display(Markdown(section))

## Introduction

Quantum computing represents a revolutionary shift in the way we process information, leveraging the principles of quantum mechanics to perform calculations at unprecedented speeds. Unlike classical computers, which use bits as the smallest unit of data, quantum computers utilize qubits that can exist in multiple states simultaneously. This unique property allows quantum systems to solve complex problems that are currently intractable for classical machines.

In recent years, the field of quantum computing has witnessed significant advancements, with major tech companies, startups, and research institutions investing heavily in the development of quantum technologies. These efforts have led to breakthroughs in quantum algorithms, error correction, and hardware improvements, bringing us closer to realizing the full potential of quantum computing.

As we look ahead to the next few years, the outlook for quantum computing is both exciting and uncertain. With ongoing research and development, we can expect to see more practical applications emerge, particularly in fields such as cryptography, materials science, and artificial intelligence. However, challenges remain, including the need for more robust quantum systems and the development of a skilled workforce to drive innovation in this cutting-edge domain.

In this article, we will explore the current state of quantum computing, the key players in the industry, and the anticipated advancements that may shape the future of this transformative technology.

## Current Developments in Quantum Computing

### Advancements in Qubit Technology
Recent years have seen remarkable progress in qubit technology, which is the fundamental building block of quantum computers. Researchers are exploring various types of qubits, including superconducting qubits, trapped ions, and topological qubits, each offering unique advantages in terms of coherence time and error rates. Companies like IBM and Google are pushing the boundaries of qubit count and fidelity, with IBM's roadmap aiming for a 1,000-qubit processor by 2023. These advancements are crucial for building more powerful and reliable quantum systems.

### Quantum Algorithms and Software
The development of quantum algorithms is another area of rapid advancement. Researchers are focusing on creating algorithms that can leverage quantum computing's unique capabilities to solve complex problems more efficiently than classical computers. Notable algorithms, such as Shor's algorithm for factoring large numbers and Grover's algorithm for searching unsorted databases, are being refined and tested. Additionally, software platforms like Qiskit and Cirq are making it easier for developers to create and run quantum algorithms, fostering a growing ecosystem of quantum software development.

### Industry Investments and Collaborations
The quantum computing landscape is witnessing a surge in investments and collaborations across industries. Tech giants, startups, and academic institutions are increasingly partnering to accelerate research and development. Companies like Microsoft and Amazon are investing heavily in quantum research, while venture capital funding for quantum startups has reached unprecedented levels. This influx of resources is not only driving innovation but also fostering a collaborative environment that is essential for overcoming the challenges facing quantum computing.

## Challenges Facing Quantum Computing

Quantum computing holds immense promise, but it is not without its challenges. As researchers and companies strive to unlock the full potential of this technology, several key obstacles must be addressed.

### Technical Challenges
One of the primary hurdles in quantum computing is the technical complexity involved in building and maintaining quantum systems. Quantum bits, or qubits, are highly sensitive to their environment, which can lead to errors in computation. Achieving stable qubits that can maintain coherence long enough to perform calculations is a significant challenge. Additionally, the development of error correction methods and robust quantum algorithms is crucial for practical applications.

### Scalability Issues
Scalability is another major concern in the field of quantum computing. While small-scale quantum processors have been developed, scaling these systems to a level where they can outperform classical computers remains a daunting task. The integration of a larger number of qubits while maintaining their coherence and minimizing error rates is a complex engineering problem. Researchers are exploring various architectures, such as superconducting qubits and trapped ions, but a universally scalable solution has yet to be realized.

### Regulatory and Ethical Considerations
As quantum computing technology advances, it raises important regulatory and ethical questions. The potential for quantum computers to break current encryption methods poses significant risks to data security and privacy. Policymakers must consider how to regulate the use of quantum technology to prevent misuse while fostering innovation. Additionally, ethical considerations surrounding the implications of quantum computing on society, such as job displacement and access to technology, must be addressed to ensure equitable benefits from these advancements.

## Future Outlook and Predictions

### Short-term Predictions
In the next few years, we can expect significant advancements in quantum hardware and software. Companies like IBM, Google, and startups such as Rigetti Computing are racing to improve qubit coherence times and error rates. By 2025, we may see the first commercially viable quantum computers capable of solving specific problems faster than classical computers. Additionally, the development of quantum algorithms will continue to evolve, with a focus on optimization and machine learning applications.

### Long-term Vision
Looking further ahead, the vision for quantum computing is ambitious. By the end of the decade, we could witness the emergence of quantum supremacy in various fields, including cryptography, materials science, and pharmaceuticals. The integration of quantum computing with classical systems will likely lead to hybrid models that leverage the strengths of both technologies. Furthermore, as quantum networks develop, we may see the establishment of a quantum internet, enabling secure communication and distributed quantum computing resources.

### Potential Applications
The potential applications of quantum computing are vast and transformative. In finance, quantum algorithms could optimize trading strategies and risk assessments. In healthcare, they could accelerate drug discovery and personalized medicine by simulating molecular interactions at unprecedented speeds. Other fields, such as logistics, artificial intelligence, and climate modeling, stand to benefit significantly from the computational power of quantum systems. As research progresses, we will likely uncover new applications that we cannot yet foresee, further expanding the impact of quantum technology on society.

## Conclusion

In summary, the current state of quantum computing is marked by significant progress and innovation, with major advancements in hardware, algorithms, and applications. As we look to the next few years, the potential for quantum technologies to revolutionize industries such as cryptography, materials science, and pharmaceuticals is becoming increasingly tangible. However, challenges such as error correction, scalability, and the need for a skilled workforce remain critical hurdles that must be addressed.

The outlook for quantum computing is promising, with ongoing investments from both public and private sectors driving research and development. As we continue to explore the capabilities of quantum systems, it is essential to foster collaboration between academia, industry, and government to ensure that the benefits of this transformative technology are realized. The journey of quantum computing is just beginning, and its future holds the potential to reshape our understanding of computation and problem-solving in ways we are only beginning to imagine.